# Procedure for splitting the Data in Train and Validation sets

**Merge all Data**

In [2]:
import pandas as pd

# URLs
base = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/"
df_umsatz = pd.read_csv(base + "umsatzdaten_gekuerzt.csv")
df_wetter = pd.read_csv(base + "wetter.csv")
df_kiwo = pd.read_csv(base + "kiwo.csv")

# Datum vereinheitlichen
for df in [df_umsatz, df_wetter, df_kiwo]:
    df["Datum"] = pd.to_datetime(df["Datum"])

# Merge
df_all = df_umsatz.merge(df_wetter, on="Datum", how="outer") \
                  .merge(df_kiwo, on="Datum", how="outer")

df_all.info()
print(df_all.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10119 entries, 0 to 10118
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   9334 non-null   float64       
 1   Datum                10119 non-null  datetime64[ns]
 2   Warengruppe          9334 non-null   float64       
 3   Umsatz               9334 non-null   float64       
 4   Bewoelkung           10048 non-null  float64       
 5   Temperatur           10103 non-null  float64       
 6   Windgeschwindigkeit  10103 non-null  float64       
 7   Wettercode           7581 non-null   float64       
 8   KielerWoche          250 non-null    float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 711.6 KB
Index(['id', 'Datum', 'Warengruppe', 'Umsatz', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode', 'KielerWoche'],
      dtype='object')


# Split the Data set by dates

training set 01.07.2013 - 31.07.2017  
val set 01.08.2017 - 31.07.2018  
test set 01.08.2018 -31.07.2019  

In [8]:
df_all = df_all.sort_values(by="Datum")

# look at the first rows
#print(df_all.head())

# define data thresshold
train_end_date = "31.07.2017"
validation_end_date = "31.07.2018"


# convert threshold dates to datetime
df_all["Datum"] = pd.to_datetime(df_all["Datum"], format="%Y-%m-%d")
train_end = pd.to_datetime(train_end_date, format="%d.%m.%Y")
validation_end = pd.to_datetime(validation_end_date, format="%d.%m.%Y")

# split data
train = df_all[df_all["Datum"] <= train_end]
validation = df_all[(df_all["Datum"] > train_end) & (df_all["Datum"] <= validation_end)]
test = df_all[df_all["Datum"] > validation_end]

print(test.head())

# check dimensions
print(f"Train shape: {train.shape}")
print(f"Validation shape: {validation.shape}")
print(f"Test shape: {test.shape}")



      id      Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
9766 NaN 2018-08-01          NaN     NaN         0.0     23.7625   
9767 NaN 2018-08-02          NaN     NaN         0.0     26.1875   
9768 NaN 2018-08-03          NaN     NaN         1.0     27.6625   
9769 NaN 2018-08-04          NaN     NaN         4.0     25.1375   
9770 NaN 2018-08-05          NaN     NaN         7.0     21.3000   

      Windgeschwindigkeit  Wettercode  KielerWoche  
9766                 10.0         0.0          NaN  
9767                 10.0         0.0          NaN  
9768                 10.0         0.0          NaN  
9769                 12.0         NaN          NaN  
9770                 14.0        61.0          NaN  
Train shape: (7917, 9)
Validation shape: (1849, 9)
Test shape: (353, 9)


## Define a simple linear model equation and conduct a linear regression using the training data


In [4]:
import statsmodels.formula.api as smf

mod = smf.ols(formula="Umsatz ~ Warengruppe", data=train)
mod = mod.fit()

# check the summary
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     27.08
Date:                Thu, 22 May 2025   Prob (F-statistic):           2.01e-07
Time:                        15:42:28   Log-Likelihood:                -48051.
No. Observations:                7493   AIC:                         9.611e+04
Df Residuals:                    7491   BIC:                         9.612e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     227.7143      3.921     58.076      

In [ ]:
import pandas as pd

# Beispiel-Daten manuell definieren
data2 = {
    "Datum": pd.to_datetime(["2018-08-01"] * 6),
    "Warengruppe": [1, 2, 3, 4, 5, 6],
    "Umsatz": [120.0, 85.5, 99.9, 130.2, 110.0, 95.3],
    "Bewoelkung": [0.0, 1.0, 2.0, 3.0, 2.0, 1.0],
    "Temperatur": [24.0, 24.0, 24.0, 24.0, 24.0, 24.0],
    "Windgeschwindigkeit": [10.0, 10.0, 10.0, 10.0, 10.0, 10.0],
    "Wettercode": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    "KielerWoche": [0, 0, 0, 0, 0, 0],
}

# DataFrame erstellen
df_sample = pd.DataFrame(data2)

# ID erzeugen im Format yyyymmdd,WARGRUPPE
df_sample["id"] = df_sample["Datum"].dt.strftime("%Y%m%d") + "," + df_sample["Warengruppe"].astype(str)

# Spaltenreihenfolge anpassen
df_sample = df_sample[["id", "Datum", "Warengruppe", "Umsatz", "Bewoelkung",
                       "Temperatur", "Windgeschwindigkeit", "Wettercode", "KielerWoche"]]

# Ausgabe
print(df_sample)

           id      Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0  20180801,1 2018-08-01            1   120.0         0.0        24.0   
1  20180801,2 2018-08-01            2    85.5         1.0        24.0   
2  20180801,3 2018-08-01            3    99.9         2.0        24.0   
3  20180801,4 2018-08-01            4   130.2         3.0        24.0   
4  20180801,5 2018-08-01            5   110.0         2.0        24.0   
5  20180801,6 2018-08-01            6    95.3         1.0        24.0   

   Windgeschwindigkeit  Wettercode  KielerWoche  
0                 10.0         0.0            0  
1                 10.0         0.0            0  
2                 10.0         0.0            0  
3                 10.0         0.0            0  
4                 10.0         0.0            0  
5                 10.0         0.0            0  


## Use Model or prediction

In [10]:
# Vorhersage für die Beispiel-Daten
pred = mod.predict(df_sample)
print(pred)

0    221.762590
1    215.810925
2    209.859260
3    203.907595
4    197.955930
5    192.004265
dtype: float64
